In [3]:
!pip uninstall umap
!pip install umap-learn
!pip install trimap

Found existing installation: umap 0.1.1
Uninstalling umap-0.1.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/umap-0.1.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/umap/*
Proceed (y/n)? y
  Successfully uninstalled umap-0.1.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 9.6 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=b02e832448a446bad7be394e9a7b3600cb3ca52db54d2029ea7af33eb6a2d49c
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=8219003e94ea27361cf296b5839e049b9f4c9c028428224d7b20fb440fcf7e89
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Succ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code')
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master')
%load_ext autoreload
%autoreload 2
## Importing libraries

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import math
import numpy as np
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from scipy.spatial.distance import pdist
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.models.glyphs import Text
from bokeh.layouts import row
from bokeh.io import export_png
output_notebook()

from sklearn.neighbors import NearestNeighbors

## Importing local Python files

sys.path.append('../') 
from src import Preprocessing, Distances, Explanation, DR_algorithms, genericMethods
from src.GAPS_tabular import GapsExplainer
from src.GAPS_Explanation import get_local_explanations_for_GAPS, compute_local_divergences

## Defining input paths (Update the Data_path attribute accordingly)

Data_path = "/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master/data/"

Mounted at /content/drive


### Step 1: Load and Pre-process Dataset

#### a. Load Dataset

In [16]:
df = pd.read_csv(Data_path+'BCancer.csv')
df = df.fillna(0)
#df = df.sample(100)
df.reset_index(inplace= True, drop= True)
print(df.shape)

(99, 31)


#### b. Identify Intrinsic Dimensionality

In [3]:
## Identify intrinsic dimensionality using: the Maximum likelihood intrinsic dimensionality estimator

int_dim = Preprocessing.repeated(Preprocessing.intrinsic_dim_scale_interval, 
                             df.values, 
                             mode='bootstrap', 
                             nb_iter=500, # nb_iter for bootstrapping
                             verbose=1, 
                             k1=10, k2=20)
print ("Intrinsic dimensionality:", int_dim)

100%|██████████| 500/500 [00:17<00:00, 28.71it/s]

Intrinsic dimensionality: 1


#### c. Set Features and Target

In [4]:
model_features, target = Preprocessing.set_features_and_target(df)

y = (df[target].values.reshape(-1, ))
X_df = pd.DataFrame(df, columns=model_features)

Features:  ['radius_mean', 'texture_mean', 'perim_mean', 'area_mean', 'smooth_mean', 'compact_mean', 'concave_mean', 'concv_p_mean', 'symetry_mean', 'frac_dim_mean', 'radius_se', 'texture_se', 'perim_se', 'area_se', 'smooth_se', 'compact_se', 'concavity_se', 'concave_p_se', 'symmetry_se', 'frac_dim_se', 'radius_worst', 'texture_worst', 'perim_worst', 'area_worst', 'smooth_worst', 'compact_worst', 'concave_worst', 'concv_p_wrst', 'symetry_worst', 'frac_dim_worst']
Target:  ['diagnosis']


#### d. Identify Categorical Features

In [5]:
X_transformed, categorical_features, numeric_features, categorical_names = Preprocessing.identify_and_transform_features(df, model_features)

### Step 2: Run Dimensionality Reduction

In [6]:
ld_embedding = DR_algorithms.run_DR_Algorithm("tSNE", X_transformed)

In [ ]:
from TorchSNE import TorchSNE
ld_embedding = TorchSNE(verbose=True, n_iter = 300).fit_transform(X_transformed)

In [ ]:
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Defaulter', 1: 'Non-defaulter'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding[:,0],
            y=ld_embedding[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

### Step 3: Obtain Representative Data-subset and Select a Single Subset

In [7]:
print(X_df.shape)
print(X_df)

(99, 30)
    radius_mean  texture_mean  perim_mean  area_mean  smooth_mean  \
0         18.63         25.11      124.80     1088.0      0.10640   
1         11.84         18.70       77.93      440.6      0.11090   
2         17.02         23.98      112.80      899.3      0.11970   
3         19.27         26.47      127.90     1162.0      0.09401   
4         16.13         17.88      107.00      807.2      0.10400   
..          ...           ...         ...        ...          ...   
94        13.37         16.39       86.10      553.5      0.07115   
95        13.85         17.21       88.44      588.7      0.08785   
96        13.61         24.69       87.76      572.6      0.09258   
97        19.00         18.91      123.40     1138.0      0.08217   
98        15.10         16.39       99.58      674.5      0.11500   

    compact_mean  concave_mean  concv_p_mean  symetry_mean  frac_dim_mean  \
0        0.18870       0.23190       0.12440        0.2183        0.06197   
1       

In [8]:
rep_subset = genericMethods.generate_representative_subset(20, X_df)

In [9]:
Rep_order = ["Density and dissimilarity based subset", "Cluster based subset"]

for index in range(0,len(rep_subset)):
    print(Rep_order[index]+" : "+''.join(str(rep_subset[index])))

Density and dissimilarity based subset : [85, 51, 81, 50, 43, 48, 19, 47, 36, 45, 93, 40, 37, 35, 83, 34, 29, 8, 71, 52]
Cluster based subset : [44, 18, 49, 58, 85, 51, 54, 28, 39, 72, 12, 0, 57, 55, 15, 91, 47, 97, 48, 3]


In [10]:
## Calculate coverage of density and dissimilarity based subset
    
genericMethods.compute_coverage(X_df, rep_subset[0])

Coverage of density and dissimilarity based subset is: 86.87%


In [11]:
## user selects a subset

data_instance_numbers = rep_subset[0]

### Step 4: Initiate and Execute GAPS Explainer

#### a. Initiate Explainer

In [15]:
print(X_df.shape)
print(len(data_instance_numbers))

(99, 30)
20


In [12]:
explainer = GapsExplainer(X_df.values,
                    feature_names=model_features,
                    class_names=target,
                    categorical_features=categorical_features,
                    categorical_names=categorical_names,
                    discretize_continuous=False,
                    discretizer='quartile',
                    random_state=42)

In [19]:
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd, local_feature_contributions, local_feature_contributions_embd, neighbors_local, neighbors_embd_local = explainer.generate_perturbed_neighborhood_global(
                                                                                X_df.values,
                                                                                X_transformed,
                                                                                ld_embedding,
                                                                                data_instance_numbers,
                                                                                model_features,
                                                                                categorical_features,
                                                                                numeric_features,
                                                                                nbrs=5,
                                                                                num_features=5,
                                                                                num_samples = 1,
                                                                                verbose = True)

num_samples = 1


100%|██████████| 20/20 [00:01<00:00, 19.96it/s]


#### b. Explain the Point in Original Data and in Embedding

In [31]:
sorted_index_combinations, sorted_index_combinations_embd = get_local_explanations_for_GAPS(X_df, ld_embedding, data_instance_numbers, local_feature_contributions)
local_divergences = compute_local_divergences(neighbors_local, neighbors_embd_local, local_feature_contributions, local_feature_contributions_embd)

In [30]:
corr_feat_dist, feature_dict, sorted_index_combinations = Explanation.explain_point_global(oversampled_data, 
                                                                               model_features, 
                                                                               categorical_features, 
                                                                               numeric_features, verbose = True)

Step 1/4


100%|██████████| 120/120 [00:00<00:00, 11991.72it/s]


Step 2/4
Step 3/4


100%|██████████| 7140/7140 [00:00<00:00, 12785.81it/s]

Step 4/4


In [18]:
corr_feat_dist_embd, feature_dict_embd, sorted_index_combinations_embd = Explanation.explain_point_global(oversampled_data_embd, 
                                                                                              model_features, 
                                                                                              categorical_features, 
                                                                                              numeric_features)

Step 1/4
Step 2/4
Step 3/4
Step 4/4


In [19]:
from sklearn.manifold import Isomap
int_dim = 2
unsquareform = lambda a: a[np.nonzero(np.triu(a, 1))]

temp_data = np.zeros((len(data_instance_numbers),local_feature_contributions.shape[1]))
temp_data_embd = np.zeros((len(data_instance_numbers),int_dim))

for index in range(0,len(data_instance_numbers)):
    temp_data[index] = X_df.values[index]
for index in range(0,int_dim):
    temp_data_embd[index] = ld_embedding[index]

iso_embd = Isomap(n_components=2, n_neighbors =1)

X_trans = iso_embd.fit_transform(temp_data)        
distance_matrix = iso_embd.dist_matrix_
iso_embd.dist_matrix_[iso_embd.dist_matrix_ == 0] = -9999
distances = unsquareform(iso_embd.dist_matrix_)

embd_trans = iso_embd.fit_transform(temp_data_embd)        
distance_matrix_embd = iso_embd.dist_matrix_
iso_embd.dist_matrix_[iso_embd.dist_matrix_ == 0] = -9999
distances_embd = unsquareform(iso_embd.dist_matrix_)

index_combinations=[]
for row in range(0,len(distance_matrix)):
    for col in range(row+1,len(distance_matrix[0])):
        index_combinations.append([row,col])

sorted_indexes = np.argsort(distances)
sorted_index_combinations = []

sorted_indexes_embd = np.argsort(distances_embd)
sorted_index_combinations_embd = []

for index in sorted_indexes_embd:
    sorted_index_combinations_embd.append(index_combinations[index])

for index in sorted_indexes:
    sorted_index_combinations.append(index_combinations[index])

#print(distances)
#print(distances_embd)
print("Original Order of Geodesic Distances\n")
print(sorted_index_combinations)
print("#################################################################################################\n")
print("After Transformation Order of Geodesic Distances\n")
print(sorted_index_combinations_embd)

Original Order of Geodesic Distances

[[14, 17], [9, 14], [0, 12], [10, 17], [9, 19], [9, 17], [10, 14], [4, 5], [14, 19], [6, 10], [7, 18], [9, 10], [17, 19], [12, 15], [11, 18], [1, 13], [6, 17], [2, 5], [10, 13], [0, 15], [10, 19], [6, 14], [6, 8], [7, 19], [13, 17], [6, 9], [0, 3], [13, 14], [7, 11], [2, 4], [6, 13], [3, 12], [7, 9], [6, 19], [9, 13], [8, 10], [1, 10], [7, 14], [18, 19], [7, 17], [13, 19], [8, 17], [1, 17], [8, 14], [9, 18], [2, 15], [1, 14], [7, 10], [3, 15], [1, 6], [1, 18], [8, 9], [14, 18], [1, 9], [11, 16], [17, 18], [8, 13], [11, 19], [8, 19], [1, 19], [6, 7], [10, 18], [2, 12], [9, 11], [1, 7], [5, 15], [7, 13], [1, 11], [0, 2], [13, 18], [11, 14], [16, 18], [11, 17], [4, 13], [1, 8], [6, 18], [4, 15], [10, 11], [5, 6], [5, 12], [7, 16], [7, 8], [5, 13], [11, 13], [0, 5], [1, 4], [6, 11], [4, 6], [4, 10], [5, 10], [4, 12], [2, 3], [8, 18], [4, 17], [5, 17], [0, 4], [2, 6], [1, 5], [4, 14], [5, 14], [2, 13], [5, 8], [4, 9], [5, 9], [16, 19], [2, 10], [4, 19],

#### c. Plot Feature Contribution in Original Data and in Embedding

In [20]:
bar_color = []
y_offset_val = []
label = []
for item in feature_dict.values():
    label.append("{0:.2f}".format(item))
    if item<0:
        bar_color.append('#e34a33')
        y_offset_val.append(item+15)
    else:
        bar_color.append('#2ca25f')
        y_offset_val.append(item)
        
        
source = ColumnDataSource(
        data=dict(
            x_val = list(feature_dict.keys()),
            y_val = list(feature_dict.values()),
            y_offset = y_offset_val,
            labels =  label,
            color = bar_color
        )
    )

p = figure(x_range=list(feature_dict.keys()), plot_height=420, plot_width=980, title="Feature Influences", x_axis_label = "Features",
        y_axis_label = "Contribution", toolbar_location=None, tools="")

glyph = Text(x="x_val", y="y_val", text='labels', x_offset=-10, y_offset='y_offset', text_font_size="9pt", text_color="black")


p.vbar(x="x_val", top="y_val", color="color", width=0.9, source=source)
p.xaxis.major_label_orientation = math.pi/3
p.xgrid.grid_line_color = None
p.y_range.start = np.min(list(feature_dict.values()))-0.0001
p.add_glyph(source, glyph)

#export_png(p, filename="Feature-Influences-Original.png")
show(p)

In [21]:
bar_color = []
y_offset_val = []
label = []
for item in feature_dict_embd.values():
    label.append("{0:.2f}".format(item))
    if item<0:
        bar_color.append('#e34a33')
        y_offset_val.append(item+15)
    else:
        bar_color.append('#2ca25f')
        y_offset_val.append(item)
        
        
source = ColumnDataSource(
        data=dict(
            x_val = list(feature_dict_embd.keys()),
            y_val = list(feature_dict_embd.values()),
            y_offset = y_offset_val,
            labels =  label,
            color = bar_color
        )
    )

p = figure(x_range=list(feature_dict_embd.keys()), plot_height=420, plot_width=980, title="Feature Influences", x_axis_label = "Features",
        y_axis_label = "Contribution", toolbar_location=None, tools="")

glyph = Text(x="x_val", y="y_val", text='labels', x_offset=-10, y_offset='y_offset', text_font_size="9pt", text_color="black")


p.vbar(x="x_val", top="y_val", color="color", width=0.9, source=source)
p.xaxis.major_label_orientation = math.pi/3
p.xgrid.grid_line_color = None
p.y_range.start = np.min(list(feature_dict_embd.values()))-0.0001
p.add_glyph(source, glyph)
#export_png(p, filename="Feature-Influences-tSNE.png")
show(p)

#### d. Calculate Local Divergence

In [22]:
components, overall_divergence = Explanation.compute_global_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd, local_divergences)

In [23]:
components = components.split(",")
print("Discrepancy in Feature Influence", components[0])
print("Discrepancy in Neighborhood Content", components[1])
print("Discrepancy in Neighborhood Order", components[2])
    

Discrepancy in Feature Influence 0.7818850199836556
Discrepancy in Neighborhood Content 0.43
Discrepancy in Neighborhood Order 0.62


In [24]:
print(overall_divergence)

0.1289967495327625
